In [ ]:
import pandas as pd
import numpy as np
import folium
from re import split, findall
from vrpy import VehicleRoutingProblem
import networkx as nx
from networkx import DiGraph

In [ ]:
coppel = pd.read_csv("Coppel_vol.csv")
print(f"El tamanio del data set es de {coppel.shape}")
coppel.head()

El tamanio del data set es de (458, 23)


,Unnamed: 0,Bodega,Cedis,factura,foliocontrol,tienda,Unnamed: 5,fechaventa,fechaenrutada,ruta,...,num_ciudad,nombreciudad,numerozona,nombrezona,numerodecasa,departamento,num_codigopostal,DCF,Calle,Vol
0,0,Guadalajara,Guadalajara,0,0,0,0,01/01/2022,18/01/2022,0,...,0,CEDIS,NaN,CEDIS,0,0,0,0.0,CEDIS,0.000000
1,19773,Guadalajara,Guadalajara,702294,7223227,800,30007_702294_7223227_800,13/01/2022,18/01/2022,2,...,66,GUADALAJARA,38.0,GENERAL REAL,17,0,44400,22911.0,MARCONI,0.000006
2,19793,30007,Guadalajara,215776,7223235,1160,30007_215776_7223235_1160,02/01/2022,18/01/2022,34,...,269,TEQUILA,264.0,NUEVA XOCHILTEPEC,87,0,46470,34204.0,CERRO VIEJO,1.810000
3,19833,30007,Guadalajara,98746,7223240,1150,30007_98746_7223240_1150,17/01/2022,18/01/2022,36,...,69,ZAPOPAN,746.0,LA HACIENDITA,2341,B,45230,68010.0,SAN MARTIN,0.094000
4,19873,30007,Guadalajara,730654,7223226,800,30007_730654_7223226_800,17/01/2022,18/01/2022,1,...,66,GUADALAJARA,947.0,MIRAVALLE II,643,0,44990,33214.0,PEDRO PATINO,0.205264


In [ ]:
coppel = coppel.drop_duplicates() 
dataBaseComplete = True
if dataBaseComplete:
  columns = ["numerodecasa", "Calle" , "num_codigopostal", "nombreciudad"]
  template = "{0: 0.0f} {1}, {2: 0.0f}, {3}, Jalisco, Mexico"
  df =coppel[columns]
else: 
  columns = ["num_codigopostal", "nombreciudad"]
  template = "{0: 0.0f} {1}, Jalisco, Mexico"
  df = coppel[columns]
  df = df.drop_duplicates(subset="num_codigopostal")

querys=[]
coords = []
for i in range(len(df)):
    querys.append(template.format(*list(df.iloc[i])))
    #coords.append(g.geocode(querys[i], timeout=10))

In [ ]:
#Writter
"""with open('listfile.txt', 'w') as filehandle:
    for listitem in coords:
        filehandle.write("".join(str(listitem[1]))+"\n")"""

'with open(\'listfile.txt\', \'w\') as filehandle:\n    for listitem in coords:\n        filehandle.write("".join(str(listitem[1]))+"\n")'

In [ ]:
#Reader
coords = pd.read_csv("listfile.txt", header = None)

In [ ]:
# Color Ruta
c = coppel["ruta"][1:-1].unique()
colors = ['blue','gray','lightred','orange','beige','green','darkgreen','lightgreen',
          'darkblue','lightblue','purple','darkpurple','pink','cadetblue','lightgray','black','red','darkred']
colores = dict(zip(c, colors[:len(c)]))

In [ ]:
m = folium.Map(location=[20.8,-103.3], tiles="OpenStreetMap", zoom_start=10)
cedis = folium.Marker(
      location=[20.6489176, -103.3409776],
      icon=folium.Icon(color="red", icon="home", prefix="fa"),
      popup=f"<stong>CEDIS de Coppel</stong>",
      tooltip=f"CEDIS")
cedis.add_to(m)
for i in  range(1, 100):#len(coords)):
  marker = folium.Marker(
      location=[coords.iloc[i][0], coords.iloc[i][1]],
      icon=folium.Icon(color=colores[coppel["ruta"][i]], icon="circle", prefix="fa"),
      popup=f"<stong>{df.iloc[i][0]}, {df.iloc[i][1]}</stong>",
      tooltip=f"Location {i}")
  marker.add_to(m)
m

In [ ]:
coords.shape

(458, 2)

In [ ]:
"""Distancias = np.zeros((coords.shape[0], coords.shape[0]))
for row in range(len(Distancias)):
     for col in range(len(Distancias)):
         Distancias[row, col] = np.sqrt(np.power(coords.iloc[row][0]-coords.iloc[col][0], 2)+np.power(coords.iloc[row][1]-coords.iloc[col][1], 2))
np.savetxt("Distancias.csv", Distancias, delimiter=",")"""
from numpy import genfromtxt
Distancias = genfromtxt('Distancias.csv', delimiter=',')

# K nearest points

In [ ]:
# K nearest points
const=[[0, x] for x in range(1,len(Distancias))]
K = 49
idx = (Distancias).argsort(axis=0)[0:K+1]
checker = {1}
while len(checker) != len(idx[0]):
    K += 1
    idx = (Distancias).argsort(axis=0)[0:K+1] 
    moves, checker=const, set()
    for i in range(len(idx[0])):
        for j in range(len(idx)):
            new = sorted([i, idx[j, i]])
            if not new[0] == new[1]:
                moves.append(new)
            checker.update(new)
moves = [list(x) for x in set(tuple(x) for x in moves)]
moves = sorted(moves)
print(f"El total de requests a Google será de {2*len(moves)}, usando {K} vecinos")

El total de requests a Google será de 27806, usando 50 vecinos


# Requesting Distances from Distance Matrix

***A mí me dijeron pide y yo pedí.***      
     *-Araceli 2022, presagiando el futuro.*

In [ ]:
# WARNING: Nearly 1:30 hrs. of execution time do not uncomment
"""movesLs=[]
for i in range(len(moves)):
    origin = coords.iloc[moves[i][0]]
    destination=coords.iloc[moves[i][1]]
    if moves[i][0] == 0:
        Ori2Des=gmaps.distance_matrix(origins=tuple(origin), destinations=tuple(destination), mode = "driving", departure_time=horaS)
    else:
        Ori2Des=gmaps.distance_matrix(origins=tuple(origin), destinations=tuple(destination), mode = "driving", departure_time=hora)
    Des2Ori=gmaps.distance_matrix(origins=tuple(destination), destinations=tuple(origin), mode = "driving", departure_time=hora)
    movesLs.append(Ori2Des)
    movesLs.append(Des2Ori)"""

'movesLs=[]\nfor i in range(len(moves)):\n    origin = coords.iloc[moves[i][0]]\n    destination=coords.iloc[moves[i][1]]\n    if moves[i][0] == 0:\n        Ori2Des=gmaps.distance_matrix(origins=tuple(origin), destinations=tuple(destination), mode = "driving", departure_time=horaS)\n    else:\n        Ori2Des=gmaps.distance_matrix(origins=tuple(origin), destinations=tuple(destination), mode = "driving", departure_time=hora)\n    Des2Ori=gmaps.distance_matrix(origins=tuple(destination), destinations=tuple(origin), mode = "driving", departure_time=hora)\n    movesLs.append(Ori2Des)\n    movesLs.append(Des2Ori)'

In [ ]:
# # Write CSV 
# """movesDf = pd.DataFrame(movesLs)
# moves[:0] = moves[::2] = moves[1::2] = moves[:]
# for i in range(0, len(moves),2):
#     moves[i] = [moves[i][1], moves[i][0]]
# movesDf.insert(loc = 0, column="Index", value= moves)
# movesDf.to_csv('last.csv')"""
# # Read CSV
movesDf = pd.read_csv("Matrix.csv")
movesDf = movesDf.drop(columns="Unnamed: 0")

In [ ]:
from networkx import DiGraph
from vrpy import VehicleRoutingProblem
G = DiGraph()

In [ ]:
movesDf = pd.read_csv("Matrix.csv")
movesDf = movesDf.drop(columns="Unnamed: 0")

In [ ]:
Distancias = np.full((coords.shape[0], coords.shape[0]), 10000)
for i in range(len(Distancias)):
    Distancias[i,i] =0

In [ ]:
for i in range(movesDf.shape[0]):
    time = split("'value': " , movesDf.iloc[i][3])
    duration = int(findall("^[0-9]*", time[2])[0])
    if len(time) > 3:
        duration += int(findall("^[0-9]*", time[3])[0]) #Traffic duration
    index = [int(x) for x in movesDf.iloc[i][0].split(",")]
    Distancias[index[0], index[1]] = duration
    if index[0] == 0:
      index[0] = "Source"
    if index[1] == 0:
      index[1] = "Sink"
    G.add_edge(index[0], index[1], cost=int(findall("^[0-9]*", time[1])[0]))
    G.edges[index[0], index[1]]["time"] = duration

In [ ]:
for i in range(1, 458):
  G.nodes[i]["demand"]= 1

In [ ]:
prob = VehicleRoutingProblem(G, load_capacity=100)
prob.num_stops = 28
prob.num_vehicles = 24
prob.use_all_vehicles = False
prob.duration = 28800

In [ ]:
prob.solve(exact=False, pricing_strategy='Hyper', max_iter=1)

INFO:vrpy.vrp:new upper bound : max num stops = 28
INFO:vrpy.vrp:Clarke & Wright solution found with value 2969826 and 36 vehicles
INFO:vrpy.vrp:Greedy solution found with value 1763325 and 19 vehicles
INFO:vrpy.vrp:iteration 0, 176102
INFO:vrpy.master_solve_pulp:total cost = 1761023.0


In [ ]:
rutas = prob.best_routes

In [ ]:
lista_rutas = list(rutas.values())

In [ ]:
for i in range(len(lista_rutas)):
    lista_rutas[i][0] = 0
    lista_rutas[i][-1] = 0

In [ ]:
rutas = []
for i in range(1,len(lista_rutas)):
    rutas.append(0)
    for j in range(1,len(lista_rutas[i])):
        if Distancias[lista_rutas[i][j-1]][lista_rutas[i][j]] == 10000:
            print(i)
            print(lista_rutas[i][j-1], lista_rutas[i][j])
        rutas[i-1] += Distancias[lista_rutas[i][j-1]][lista_rutas[i][j]]/3600

In [ ]:
rutas

[7.33,
 4.7747222222222225,
 3.9166666666666665,
 3.512500000000001,
 3.7711111111111113,
 5.0825000000000005,
 3.7405555555555554,
 4.728055555555555,
 4.612222222222222,
 5.6025,
 6.889722222222222,
 5.117777777777778,
 5.502777777777778,
 7.46388888888889,
 7.933055555555556,
 6.364444444444445,
 6.058888888888887,
 7.828055555555556]